# Chapter 4 Model Chain

- [1. Set OpenAI API Key](#1. Set OpenAI-API-Key)
- [2. Large Language Model Chain](#2. Large Language Model Chain)
- [2.1 Import Data](#2.1-Import Data)
- [2.2 Initialize Language Model](#2.2-Initialize Language Model)
- [2.3 Initialize Prompt Template](#2.3-Initialize Prompt Template)
- [2.4 Build Large Language Model Chain](#2.4-Build Large Language Model Chain)
- [2.5 Run Large Language Model Chain](#2.5-Run Large Language Model Chain)
- [2.6 Chinese Prompt](#2.6-Chinese Prompt)
- [3. Sequential Chain](#3. Sequential Chain)
- [3.1 Simple Sequential Chain](#3.1-Simple Sequential Chain)
- [3.1.1 Create Two Subchains](#3.1.1-Create Two Subchains)
- [3.1.2 Build a simple sequential chain](#3.1.2-Build a simple sequential chain)
- [3.1.3 Run a simple sequential chain](#3.1.3-Run a simple sequential chain)
- [3.1.4 Chinese prompts](#3.1.4-Chinese prompts)- [3.2 Sequence chain](#3.2-Sequence chain)
- [3.2.1 Create four subchains](#3.2.1-Create four subchains)
- [3.2.2 Combine four subchains](#3.2.2-Combining four subchains)
- [3.2.3 Chinese prompts](#3.2.3-Chinese prompts)
- [IV. Routing chain](#IV.-Routing chain)
- [4.1 Define prompt template](#4.1-Define prompt template)
- [4.2 Name and describe prompt template](#4.2-Name and describe prompt template)
- [4.3 Create corresponding target chain based on prompt template information](#4.3-Create corresponding target chain based on prompt template information--)
- [4.4 Create default target chain](#4.4-Create default target chain)
- [4.5 Define routing templates between different chains](#4.5-Define routing templates between different chains)
- [4.6 Build routing chains](#4.6-Build routing chains)
- [4.7 Create overall links](#4.7-Create overall links)
- [4.8 Ask questions](#4.8-Ask questions)
- [4.9 Chinese prompts](#4.9-Chinese promptsText Tips)

Chains typically combine a large language model (LLM) with a prompt, based on which we can perform a series of operations on text or data.

Chains can accept multiple inputs at once

For example, we can create a chain that accepts user input, formats it using a prompt template, and then passes the formatted response to the LLM. We can build more complex chains by combining multiple chains together, or by combining chains with other components.

## 1. Set up OpenAI API Key

Log in to your [OpenAI account](https://platform.openai.com/account/api-keys) to get your API Key, and then set it as an environment variable.

- If you want to set it as a global environment variable, you can refer to [Zhihu article](https://zhuanlan.zhihu.com/p/627665725).
- If you want to set it as a local/project environment variable, create a `.env` file in this file directory, open the file and enter the following content.

<p style="font-family:verdana; font-size:12px;color:green">
OPENAI_API_KEY="your_api_key" 
</p>

Replace "your_api_key" with your own API Key

In [10]:
# Download the required packages python-dotenv and openai
# If you need to view the installation process log, you can remove -q
!pip install -q python-dotenv
!pip install -q openai

In [11]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Read local/project environment variables.

# find_dotenv() finds and locates the path of the .env file
# load_dotenv() reads the .env file and loads the environment variables in it into the current running environment
# If you set a global environment variable, this line of code will have no effect.
_ = load_dotenv(find_dotenv())

# Get the environment variable OPENAI_API_KEY
openai.api_key = os.environ['OPENAI_API_KEY']  

## 2. Large Language Model Chain

The Large Language Model Chain (LLMChain) is a simple but very powerful chain, and is the basis for many of the chains we will introduce later.

### 2.1 Importing Data

In [1]:
!pip install -q pandas

In [4]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


### 2.2 Initialize the language model

In [8]:
from langchain.chat_models import ChatOpenAI 
from langchain.prompts import ChatPromptTemplate  
from langchain.chains import LLMChain   

In [13]:
# Here we set the parameter temperature to 0.0 to reduce the randomness of the generated answers.
# If you want to get different and innovative answers every time, you can try adjusting this parameter.
llm = ChatOpenAI(temperature=0.0)  

### 2.3 Initialize prompt template
Initialize prompt, this prompt will accept a variable called product. This prompt will ask LLM to generate the best name to describe the company that makes the product.

In [14]:
prompt = ChatPromptTemplate.from_template(   
    "What is the best name to describe \
    a company that makes {product}?"
)

### 2.4 Building a Large Language Model Chain

Combine the Large Language Model (LLM) and Prompt into a chain. This large language model chain is very simple and allows us to run the prompt in a sequential manner and combine it into the large language model.

In [15]:
chain = LLMChain(llm=llm, prompt=prompt)

### 2.5 Running a Large Language Model Chain
So if we have a product called "Queen Size Sheet Set", we can run it through this chain by using chain.run

In [9]:
product = "Queen Size Sheet Set"
chain.run(product)

'Royal Linens.'

You can enter any product description and see what results the chain will output

### 2.6 Chinese prompts

In [16]:
prompt = ChatPromptTemplate.from_template(   
    "描述制造{product}的一个公司的最佳名称是什么?"
)
chain = LLMChain(llm=llm, prompt=prompt)
product = "大号床单套装"
chain.run(product)

'"豪华床纺"'

## 3. Sequence Chain

### 3.1 Simple Sequential Chains

Sequential Chains are chains that execute their links in a predefined order. Specifically, we will use SimpleSequentialChain, which is the simplest type of sequential chain, where each step has an input/output and the output of one step is the input of the next step.

In [28]:
from langchain.chains import SimpleSequentialChain

In [22]:
llm = ChatOpenAI(temperature=0.9)

#### 3.1.1 Create two subchains

In [23]:
# Prompt Template 1: This prompt will accept a product and return the best name to describe the company
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [24]:

# Prompt Template 2: Accepts a company name and outputs a 20-word description of the company
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

#### 3.1.2 Building a Simple Sequential Chain
Now we can combine the two LLMChains so that we can create the company name and description in one step

In [25]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

Give an input and run the above chain

#### 3.1.3 Running a simple sequence chain

In [27]:
product = "Queen Size Sheet Set"
overall_simple_chain.run(product)



> Entering new  chain...
Royal Rest Linens
Royal Rest Linens provides high-quality and luxurious linens for a comfortable and restful night's sleep.

> Finished chain.


"Royal Rest Linens provides high-quality and luxurious linens for a comfortable and restful night's sleep."

#### 3.1.4 Chinese prompts

In [19]:
# Chinese

first_prompt = ChatPromptTemplate.from_template(   
    "描述制造{product}的一个公司的最好的名称是什么"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = ChatPromptTemplate.from_template(   
    "写一个20字的描述对于下面这个\
    公司：{company_name}的"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)


overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],verbose=True)
product = "大号床单套装"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"尺寸王床品有限公司"
尺寸王床品有限公司是一家专注于床上用品生产的公司。

> Finished chain.


'尺寸王床品有限公司是一家专注于床上用品生产的公司。'

### 3.2 Sequence Chain

When there is only one input and one output, a simple sequential chain (SimpleSequentialChain) can be implemented. When there are multiple inputs or multiple outputs, we need to use a sequential chain (SequentialChain) to implement it.

In [29]:
from langchain.chains import SequentialChain
from langchain.chat_models import ChatOpenAI    #导入OpenAI模型
from langchain.prompts import ChatPromptTemplate   #导入聊天提示模板
from langchain.chains import LLMChain    #导入LLM链。

In [30]:
llm = ChatOpenAI(temperature=0.9)

Next we will create a series of chains and then use them one by one

#### 3.2.1 Create four subchains

In [38]:
#Subchain 1

# prompt template 1: Translate into English (translate the following review into English)
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: Input: Review Output: English Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

In [39]:
#Subchain 2

# prompt template 2: Summarize the following review in one sentence
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: Input: English Review Output: Summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")


In [40]:
#Subchain 3

# prompt template 3: What language is used in the following review
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: Input: Review Output: Language
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")


In [41]:
#Subchain 4

# prompt template 4: Write a follow-up response to the following summary using specific language
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: Input: summary, language Output: follow-up reply
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")


#### 3.2.2 Combining four subchains

In [42]:
# Input: review
#Output: English review, summary, follow-up reply
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

Let's select a review and pass it through the entire chain. We can see that the original review is in French, the English review can be seen as a translation, followed by a summary based on the English review, and the final output is a continuation of the original French information.

In [43]:
review = df.Review[5]
overall_chain(review)



> Entering new  chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in the store and the taste is much better...\nOld batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the taste and consistency of the product, suspecting that either an old batch or counterfeit version is the cause.',
 'followup_message': "Réponse de suivi :\n\nCher(e) critique,\n\nNous sommes désolés d'apprendre que notre produit ne répond pas à vos attentes. Nous prenons très au sérieux les commentaires de nos clients et nous tenons à vous rassurer que nous attachons une grande importance à la qualité de nos produits.\n\nNous vérifions constamment nos processus de production afin de nous assurer que nos produits sont toujours frais et authentiques. Si vous pensez avoir reçu un produit con

#### 3.2.3 Chinese prompts

In [37]:
# Chinese

#Subchain 1
# prompt template 1: Translate into English (translate the following review into English)
first_prompt = ChatPromptTemplate.from_template(
    "把下面的评论review翻译成英文:"
    "\n\n{Review}"
)
# chain 1: Input: Review Output: English Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

#Subchain 2
# prompt template 2: Summarize the following review in one sentence
second_prompt = ChatPromptTemplate.from_template(
    "请你用一句话来总结下面的评论review:"
    "\n\n{English_Review}"
)
# chain 2: Input: English Review Output: Summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")


#Subchain 3
# prompt template 3: What language is used in the following review
third_prompt = ChatPromptTemplate.from_template(
    "下面的评论review使用的什么语言:\n\n{Review}"
)
# chain 3: Input: Review Output: Language
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")


#Subchain 4
# prompt template 4: Write a follow-up response to the following summary using specific language
fourth_prompt = ChatPromptTemplate.from_template(
    "使用特定的语言对下面的总结写一个后续回复:"
    "\n\n总结: {summary}\n\n语言: {language}"
)
# chain 4: Input: summary, language Output: follow-up reply
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


# Combine the four subchains
#Input: review Output: English review, summary, follow-up response
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)


review = df.Review[5]
overall_chain(review)



> Entering new  chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't last, it's weird. I buy the same ones from the store and the taste is much better...\nOld batch or counterfeit!?",
 'summary': "The taste is mediocre, the foam doesn't last, and there is suspicion of old batch or counterfeit.",
 'followup_message': "回复: Je suis désolé de vous entendre dire que le goût est moyen et que la mousse ne dure pas longtemps. Nous prenons ces problèmes très au sérieux. Nous allons enquêter pour vérifier s'il s'agit d'un ancien lot ou d'une contrefaçon. Votre satisfaction est notre priorité et nous ferons de notre mieux pour résoudre ce problème. Merci de nous avoir informés."}

## 4. Routing Chain

So far, we've learned about large language model chains and sequential chains. But what if we want to do something more complex?

A fairly common but basic operation is to take an input and route it to a chain, depending on what exactly that input is. If you have multiple child chains, each specialized for a specific type of input, you can make up a routing chain that first decides which child chain to pass it to, and then passes it to that chain.

A router consists of two components:

- Router Chain: The router chain itself, which is responsible for selecting the next chain to call

- destination_chains: The chains that the router chain can route to

For a concrete example, let's look at where we are routing between different types of chains, where we have different prompts:

In [28]:
from langchain.chains.router import MultiPromptChain  #导入多提示链
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [29]:
llm = ChatOpenAI(temperature=0)

### 4.1 Defining prompt templates

First, we define prompt templates that are suitable for different scenarios.

In [49]:
#The first hint is suitable for answering physics questions
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


#The second prompt is suitable for answering math problems
math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""


#The third one is suitable for answering historical questions
history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


#The fourth one is suitable for answering computer questions
computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

### 4.2 Naming and describing the prompt template

After defining these prompt templates, we can name each template and give a specific description. For example, the first physics description is suitable for answering questions about physics, and this information will be passed to the routing chain, which will then decide when to use this sub-chain.

In [27]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

LLMRouterChain (This chain uses the LLM to determine how to route things)

Here we need a **multi-tip chain**. This is a specific type of chain that is used to route between multiple different tip templates. But this is just one type of routing, we can also route between any type of chain.

A couple of the classes we are going to implement here is the large model router chain. This class itself uses the language model to route between different child chains. This is where the description and name provided above will be used.

### 4.3 Create the corresponding target chain based on the prompt template information 
The target chain is the chain called by the routing chain, and each target chain is a language model chain

In [30]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

### 4.4 Create a default target chain
In addition to the target chain, we also need a default target chain. This is a chain that is called when the router cannot decide which child chain to use. In the example above, it might be called when the input question has nothing to do with physics, mathematics, history, or computer science.

In [31]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

### 4.5 Define routing templates between different chains

This includes a description of the task to be completed and the specific format the output should be in.

Note: An example is added here based on the original tutorial, mainly because the "gpt-3.5-turbo" model is not well adapted to understand the meaning of the template, and using "text-davinci-003" or "gpt-4-0613" can work well, so more example prompts are added here to make it better to learn.
eg:
<< INPUT >>
"What is black body radiation?"
<< OUTPUT >>
```json
{{{{
"destination": string \ name of the prompt to use or "DEFAULT"
"next_inputs": string \ a potentially modified version of the original input
}}}}
```

In [32]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>

eg:
<< INPUT >>
"What is black body radiation?"
<< OUTPUT >>
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

"""

### 4.6 Building the Routing Chain
First, we create the complete router template by formatting the target defined above. This template can be used for many different types of targets.
So here, instead of just physics, math, history, and computer science, you could add a different subject like English or Latin.

Next, we create the prompt template from this template

Finally, we create the routing chain by passing in the llm and the entire routing prompt. It is important to note that there is routing output parsing here, which is important because it will help this link decide which sub-links to route between.

In [34]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

### 4.7 Create the overall link

In [35]:
#Multiple prompt chains
chain = MultiPromptChain(router_chain=router_chain,    #l路由链路
                         destination_chains=destination_chains,   #目标链路
                         default_chain=default_chain,      #默认链路
                         verbose=True   
                        )

### 4.8 Asking questions

If we ask a physical question, we expect to see it routed to the physical link

In [ ]:
# Question: What is blackbody radiation?
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation is the electromagnetic radiation emitted by a perfect black body, which absorbs all incident radiation and reflects none. It is characterized by a continuous spectrum of radiated energy that is dependent on the temperature of the body, with higher temperatures leading to more intense and shorter wavelength radiation. This phenomenon is an important concept in thermal physics and has numerous applications, ranging from understanding stellar spectra to designing artificial light sources.'

If we ask a math question, we would like to see it routed to the math link

In [22]:
# Question: What is 2+2?
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'As an AI language model, I can answer this question. The answer to 2 + 2 is 4.'

What happens if we pass it a question that has nothing to do with any of the sub-links?

Here, we asked a question about biology, and we can see that the link it chose is None. This means it will be passed to the default link, which itself is just a generic call to the language model. The language model luckily knows a lot about biology, so it can help us out.

In [40]:
# Question: Why does every cell in our body contain DNA?
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics and functions of each cell. DNA contains the instructions for the synthesis of proteins, which are essential for the structure and function of cells. Additionally, DNA is responsible for the transmission of genetic information from one generation to the next. Therefore, every cell in our body needs DNA to carry out its specific functions and to maintain the integrity of the organism as a whole.'

### 4.9 Chinese prompts

In [51]:
# Chinese
#The first hint is suitable for answering physics questions
physics_template = """你是一个非常聪明的物理专家。 \
你擅长用一种简洁并且易于理解的方式去回答问题。\
当你不知道问题的答案时，你承认\
你不知道.

这是一个问题:
{input}"""


#The second hint is suitable for answering math questions
math_template = """你是一个非常优秀的数学家。 \
你擅长回答数学问题。 \
你之所以如此优秀， \
是因为你能够将棘手的问题分解为组成部分，\
回答组成部分，然后将它们组合在一起，回答更广泛的问题。

这是一个问题：
{input}"""


#The third one is suitable for answering historical questions
history_template = """你是以为非常优秀的历史学家。 \
你对一系列历史时期的人物、事件和背景有着极好的学识和理解\
你有能力思考、反思、辩证、讨论和评估过去。\
你尊重历史证据，并有能力利用它来支持你的解释和判断。

这是一个问题:
{input}"""


#The fourth one is suitable for answering computer questions
computerscience_template = """ 你是一个成功的计算机科学专家。\
你有创造力、协作精神、\
前瞻性思维、自信、解决问题的能力、\
对理论和算法的理解以及出色的沟通技巧。\
你非常擅长回答编程问题。\
你之所以如此优秀，是因为你知道  \
如何通过以机器可以轻松解释的命令式步骤描述解决方案来解决问题，\
并且你知道如何选择在时间复杂性和空间复杂性之间取得良好平衡的解决方案。

这还是一个输入：
{input}"""

In [52]:
# Chinese
prompt_infos = [
    {
        "名字": "物理学", 
        "描述": "擅长回答关于物理学的问题", 
        "提示模板": physics_template
    },
    {
        "名字": "数学", 
        "描述": "擅长回答数学问题", 
        "提示模板": math_template
    },
    {
        "名字": "历史", 
        "描述": "擅长回答历史问题", 
        "提示模板": history_template
    },
    {
        "名字": "计算机科学", 
        "描述": "擅长回答计算机科学问题", 
        "提示模板": computerscience_template
    }
]


In [ ]:
# Chinese
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["名字"]
    prompt_template = p_info["提示模板"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['名字']}: {p['描述']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
# Chinese

# Multiple Prompt Routing Template
MULTI_PROMPT_ROUTER_TEMPLATE = """给语言模型一个原始文本输入，\
让其选择最适合输入的模型提示。\
系统将为您提供可用提示的名称以及最适合改提示的描述。\
如果你认为修改原始输入最终会导致语言模型做出更好的响应，\
你也可以修改原始输入。


<< 格式 >>
返回一个带有JSON对象的markdown代码片段，该JSON对象的格式如下：
```json
{{{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}}}
```


记住：“destination”必须是下面指定的候选提示名称之一，\
或者如果输入不太适合任何候选提示，\
则可以是 “DEFAULT” 。
记住：如果您认为不需要任何修改，\
则 “next_inputs” 可以只是原始输入。

<< 候选提示 >>
{destinations}

<< 输入 >>
{{input}}

<< 输出 (记得要包含 ```json)>>

样例:
<< 输入 >>
"什么是黑体辐射?"
<< 输出 >>
```json
{{{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}}}
```

"""

In [ ]:
#Multiple prompt chains
chain = MultiPromptChain(router_chain=router_chain,    #l路由链路
                         destination_chains=destination_chains,   #目标链路
                         default_chain=default_chain,      #默认链路
                         verbose=True   
                        )

In [36]:
#Chinese
chain.run("什么是黑体辐射？")



> Entering new MultiPromptChain chain...
physics: {'input': '什么是黑体辐射？'}
> Finished chain.


'黑体辐射是指一个理想化的物体，它能够完全吸收所有入射到它表面的辐射能量，并以热辐射的形式重新发射出来。黑体辐射的特点是其辐射能量的分布与温度有关，随着温度的升高，辐射能量的峰值会向更短的波长方向移动。这个现象被称为黑体辐射谱的位移定律，由普朗克在20世纪初提出。黑体辐射在研究热力学、量子力学和宇宙学等领域中具有重要的应用。'

In [41]:
# Chinese
chain.run("你知道李白是谁嘛?")



> Entering new MultiPromptChain chain...
History: {'input': '你知道李白是谁嘛?'}
> Finished chain.


'李白是唐朝时期的一位著名诗人。他的诗歌以豪放、奔放、自由的风格著称，被誉为“诗仙”。他的作品涉及广泛，包括山水田园、历史传说、哲理思考等多个方面，对中国古典文学的发展产生了深远的影响。'

In [37]:
# Chinese
chain.run("2 + 2 等于多少")



> Entering new MultiPromptChain chain...
math: {'input': '2 + 2 等于多少'}

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..



> Finished chain.


'2 + 2 等于 4。'

In [38]:
# Chinese
chain.run("为什么我们身体里的每个细胞都包含DNA？")



> Entering new MultiPromptChain chain...
None: {'input': '为什么我们身体里的每个细胞都包含DNA？'}
> Finished chain.


'我们身体里的每个细胞都包含DNA，是因为DNA是遗传信息的载体。DNA是由四种碱基（腺嘌呤、鸟嘌呤、胸腺嘧啶和鳞嘌呤）组成的长链状分子，它存储了生物体的遗传信息，包括个体的特征、生长发育、代谢功能等。每个细胞都需要这些遗传信息来执行其特定的功能和任务，因此每个细胞都需要包含DNA。此外，DNA还能通过复制和传递给下一代细胞和个体，以保证遗传信息的传承。'